In [2]:
import tensorflow as tf  
from tensorflow.examples.tutorials.mnist import input_data

In [4]:
#载入数据集  
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)  
  
# 输入图片是28*28  
n_inputs = 28 #输入一行，一行有28个数据  
max_time = 28 #一共28行  
lstm_size = 50 #隐层单元  
n_classes = 10 # 10个分类  
batch_size = 50 #每批次50个样本  
n_batch = mnist.train.num_examples // batch_size #计算一共有多少个批次  
print(n_batch)  
#这里的none表示第一个维度可以是任意的长度  
x = tf.placeholder(tf.float32,[None,784])  
#正确的标签  
y = tf.placeholder(tf.float32,[None,10])  
  
#初始化权值  
weights = tf.Variable(tf.truncated_normal([lstm_size, n_classes], stddev=0.1))  
#初始化偏置值  
biases = tf.Variable(tf.constant(0.1, shape=[n_classes]))  
  
  
#定义RNN网络  
def RNN(X,weights,biases):  
    # inputs=[batch_size, max_time, n_inputs]  
    inputs = tf.reshape(X,[-1,max_time,n_inputs])  
    #定义LSTM基本CELL  
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(lstm_size)  
#       final_state[0]是cell state  
#       final_state[1]是hidden_state  最后500个隐藏单元的输出结果，  
#       output 与time.major 如果是false 返回是 batch_size=50次, maxtime=28的长度，cell.output_size:500个隐藏单元  
#                                 但是time是 0到27，则cell.output_size 则表示对应时间的500个输出结果。  
#       finale_state返回：  
#           state: 包括cell_state中间的cell,hidden_state最后的结果输出。  
#           batch_size: 50次  
#           state_size: 隐藏单元个数 500  
  
    #    隐藏层的单元个数， （batch次数，个数，数据数），格式  
    outputs,final_state = tf.nn.dynamic_rnn(lstm_cell,inputs,dtype=tf.float32)  
    print(outputs[2])  
    results = tf.nn.softmax(tf.matmul(final_state[1],weights) + biases)  
    return results  
      
      
#计算RNN的返回结果  
prediction= RNN(x, weights, biases)    
#损失函数  
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y))  
#使用AdamOptimizer进行优化  
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)  
#结果存放在一个布尔型列表中  
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))#argmax返回一维张量中最大的值所在的位置  
#求准确率  
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))#把correct_prediction变为float32类型  
#初始化  
init = tf.global_variables_initializer()  
saver = tf.train.Saver()  
  
with tf.Session() as sess:  
    sess.run(init)  
    print(sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels}))  
    saver.restore(sess,'net/my_net.ckpt')  
    print(sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels}))

Extracting MNIST_data/train-images-idx3-ubyte.gz


Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


1100
Tensor("strided_slice:0", shape=(28, 50), dtype=float32)


0.1485
INFO:tensorflow:Restoring parameters from net/my_net.ckpt


InvalidArgumentError: Assign requires shapes of both tensors to match. lhs shape= [784,10] rhs shape= [50,10]
	 [[Node: save_1/Assign = Assign[T=DT_FLOAT, _class=["loc:@Variable"], use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](Variable, save_1/RestoreV2/_41)]]

Caused by op 'save_1/Assign', defined at:
  File "D:\Program Files (x86)\anaconda\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "D:\Program Files (x86)\anaconda\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "D:\Program Files (x86)\anaconda\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "D:\Program Files (x86)\anaconda\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "D:\Program Files (x86)\anaconda\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "D:\Program 